In [1]:
!pip install soundfile
!pip install pandas
!pip install scipy
!pip install librosa
!pip install pykaldi
!pip install torchaudio

from models import BiLSTMModel
from dataset import preprocess_viseme, VisemeDataset
from audio import load_and_pad_audio
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader


import numpy as np
import soundfile as sf
import scipy
from scipy import fft
import functools
from torch import nn
import librosa
import math
import torchaudio 
import math
import yaml
from tensorflow_tts.inference import AutoConfig
import json


from dataset import preprocess_viseme, VisemeDataset

torch.__version__

device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Using {} device'.format(device))

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


################################################################################
### WARNING, path does not exist: KALDI_ROOT=/mnt/matylda5/iveselyk/Tools/kaldi-trunk
###          (please add 'export KALDI_ROOT=<your_path>' in your $HOME/.profile)
###          (or run as: KALDI_ROOT=<your_path> python <your_script>.py)
################################################################################

/home/hydroxide/.local/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's

Using cuda device


\---S1---/\---S2---/\---S3---/
WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
____________\---V1---/
_____________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

             \---S1---/\---S2---/\---S3---/
             WWWWWWWWWWWWWWWWWWWWWWWWWWWWWW
______________________\---V2---/
_____________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAA


A = audio sample
V1, V2, etc = viseme 1, viseme 2, etc (aka "frame", but not in the MFCC sense)
W = a window of audio samples that will be used as input (aka "frame" in the MFCC sense. to avoid confusion, we refer to this as the "window" and the viseme as the "frame")
S1, S2, S3 = STFT of a window 
num_frames == num_windows

audio_bins_per_window = the number of S per W
samples_per_bin = the length of each S (i.e. number of As)
bin_hop_length = the number of A between S1 and S2 (in the picture above, hop_length == len(S1) == len(S2) == samples_per_bin

in practice, we calculate as such:
          /-----------V2--------------\
/--------------V1------------\
\---S1---/\---S2---/\---S3---/\---S4---/\---S5---/\---S6---/
_______________AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA

then at V1, we take {S1,S2,S3}, at V2 we take {S4,S5,S6}, etc
The STFT 
STFT hop length should therefore equal the length of a single viseme frame, in samples

In [2]:
# let num_stft_frames be the number of STFT frames per audio sample 
# num_stft_frames == (audio_len / n_fft)
# (assuming FFT window length == hop length == n_ffts)
# coeffs will be B x num_stft_frames x num_mels
# every 
# if we've calculated the FFT size correctly, the hop length will be half 
# so stft_frames_per_window should be odd

def coeffs_to_windows(coeffs, num_frames, stft_frames_per_window, n_mels):
    
    output = np.zeros((num_frames, stft_frames_per_window*n_mels))
    
    # the number of frames to hop is just half the number of STFT frames per window
    hop_len = int(stft_frames_per_window / 2)
    
    for i in range(num_frames):
        c = coeffs[:, (i*hop_len):(i*hop_len)+stft_frames_per_window]
        # padding
        if(c.shape[1] < stft_frames_per_window):
            c = np.pad(c, [(0,0), (0, stft_frames_per_window - c.shape[1])], constant_values=0)
        output[i, :] = np.reshape(c, stft_frames_per_window*n_mels)
    return output


# featurize audio in exactly the same way as TensorflowTTS
# this enables mels from the synthesis step to be reused for viseme prediction
def kaldi_mels(filepath, 
               config=None):
    audio = load_and_pad_audio(filepath, config["sampleRate"], config["paddedAudioLength"])
    audio = torch.unsqueeze(torch.tensor(audio, dtype=torch.float32), dim=0)
    
    fbank_framelength = 1000 * config["fftSize"] / config["sampleRate"];
    fbank_frameshift =fbank_framelength #1000 *config["hopSize"] / config["sampleRate"]
    
    output = torchaudio.compliance.kaldi.fbank(audio,
                                  frame_length=fbank_framelength,
                                  frame_shift =fbank_frameshift,
                                  num_mel_bins=config["numMels"],
                                  high_freq=config["fmax"], 
                                  low_freq=config["fmin"], 
                                  sample_frequency=config["sampleRate"],
                                  use_energy=False,
                                  htk_compat=False,
                                  use_log_fbank=True,
                                  use_power=False,
                                  energy_floor=0.0,
                                  window_type='povey'
                                 )
    num_frames = (config["sampleRate"] * config["paddedAudioLength"]) // config["frameLength"]
    output = output.transpose(0,1)
    #print(f"output shape {output.size()} and num_frames {num_frames}, stft frames perindow {config['stftFramesPerWindow']}")
    output = coeffs_to_windows(output, num_frames,  config["stftFramesPerWindow"], config["numMels"])


    return torch.tensor(output,dtype=torch.float32).to(device), audio.size()[0], [1] * audio.size()[0]
#kaldi_mels("tmp/original.wav", model_config)

In [3]:
from collections import OrderedDict
import pandas as pd

mappings = OrderedDict()
mappings["MouthClose"] = "A37_Mouth_Close"
mappings["MouthFunnel"] = "A29_Mouth_Funnel"
mappings["MouthPucker"] = "A30_Mouth_Pucker"
#mappings["JawOpen"] = "Mouth_Open"
mappings["JawOpen"] = "Merged_Open_Mouth"
mappings["EyeBlinkLeft"] = "Eye_Blink_L"
mappings["EyeBlinkRight"] = "Eye_Blink_R"
mappings["EyeSquintLeft"] = "Eye_Squint_L"
mappings["EyeSquintRight"] = "Eye_Squint_R"
mappings["BrowDownLeft"] = "Brow_Drop_L"
mappings["BrowDownRight"] = "Brow_Drop_R"
mappings["MouthUpperUpLeft"] = "Mouth_Snarl_Upper_L"
mappings["MouthUpperUpRight"] = "Mouth_Snarl_Upper_R"
mappings["MouthLowerDownLeft"] = "Mouth_Snarl_Lower_L"
mappings["MouthLowerDownRight"] = "Mouth_Snarl_Lower_R"
mappings["MouthRollUpper"] = "Mouth_Top_Lip_Under" # not exact

#mappings["CheekBlow"] = "Cheek_Blow_L", "Cheek_Blow_R"
#Nose_Scrunch = Nose_Sneer_L + Nose_Sneer_R
print(list(mappings.keys()))
# config for the input blendshapes
source_config = {
    # source framerate for raw viseme label input
    "framerate":59.97,
    # we need to map the blendshapes from the incoming CSV from LiveLinkFace to CC3 blendshapes
    # unfortunately not 1-to-1, but seems to work well enough
    "mappings":mappings
}
 
# config for the viseme model
model_config = {}
# the filename for the trained model that we will export below
model_config["modelPath"] = "bilstm.tflite"
# the blendshapes that will be predicted
# we also need to export this so the gltf animator can match the model outputs indices to morph target indices
model_config["targetNames"] = list(mappings.values()) #["A37_Mouth_Close", "A29_Mouth_Funnel", "A30_Mouth_Pucker", "Mouth_Open"]

# actual framerate to use for viseme labels. 
# raw labels will be resampled/transformed (either averaged or simply dropped).
model_config["frameRate"] = source_config["framerate"] / 3
# the sample rate that audio will be resampled to
model_config["sampleRate"] = 22050

# the duration of a viseme frame is (1 / target_framerate) seconds
model_config["frameLength"] = math.ceil(model_config["sampleRate"] * (1 / model_config["frameRate"]))
# all audio will be padded to the following size
model_config["paddedAudioLength"] = 10
# the raw input for each viseme frame will be an audio window of size X 
# the middle sample of the viseme frame is aligned with the middle sample of the audio window
# this means, at the nominal "anchor sample" of the viseme frame, there will be 
# X/2 samples to the left and X/2 samples to the right
model_config["windowLength"] = int(1 * model_config["sampleRate"]) 

# this raw audio input will then be transformed into a number of STFT frames/coefficients
# each viseme frame will have this number of STFT frames, which will be the actual input at each timestep
# Since audio windows overlap, we won't want to waste cycles repeatedly computing the STFT across the whole audio sequence
# So in practice, we pre-calculate the STFT for the whole sequence, then just sub-sample the coefficients at each timestep
# when assigning STFT frames, the hop length will then just be half this value
model_config["stftFramesPerWindow"] = 20

model_config["hopSize"] = int(model_config["windowLength"] // model_config["stftFramesPerWindow"])
model_config["numMels"] = 30
model_config["fmin"] = 1000
model_config["fmax"] = 6000
model_config["fftSize"] = 512

# load the TTS config so we can match the same parameters 
# this may override some of the parameters set above
USE_TFTTS_CONFIG=False

USE_KALDI_FBANK=True


if USE_TFTTS_CONFIG:
    with open("/mnt/hdd_2tb/home/hydroxide/projects/TensorFlowTTS/preprocess/baker_preprocess.yaml", "r") as f:
        tftts_config = yaml.safe_load(f)
        model_config["fftSize"] = tftts_config["fft_size"]
        model_config["numMels"] = tftts_config["num_mels"]       
        if "window_length" in tftts_config:
            assert(tftts_config["window_length"] == model_config["windowLength"])
        
        model_config["window"] = tftts_config["window"]
        model_config["fmin"] = tftts_config["fmin"]
        model_config["fmax"] = tftts_config["fmax"]
        assert(tftts_config["sampling_rate"] == model_config["sampleRate"])
        
        #config={"num_mels":80, "sampling_rate":22050,"fmin":80,"fmax":6000, "window":"hann", "fft_size":512, "hop_size":hop_size})

#seq_length = math.ceil((pad_len_in_secs * resample_to) / frame_len)

# save the config so we can pass
with open("output/viseme_model.json", "w",encoding="utf-8") as outfile:
    json.dump(model_config, outfile)
    
process_audio = None

if USE_KALDI_FBANK:
    process_audio = functools.partial(kaldi_mels, 
                                          config=model_config)  
elif USE_TFTTS_CONFIG:
    process_audio = functools.partial(tftts_mels, 
                                          config=model_config)
else:
    num_mels=39
    process_audio = functools.partial(stft, model_config);
        #viseme_frame_len_in_samples=viseme_frame_len_in_samples, # this refers to the size of the viseme/audio window,
        #audio_window_in_samples=audio_window_in_samples, # TODO - update these
        #stft_frames_per_window=stft_frames_per_window,
        #resample_to=resample_to, 
        #pad_len_in_secs=pad_len_in_secs,
        #n_mels=num_mels)
        
batch_size = 5
seq_length = 199
text_pad_len=seq_length

process_viseme = functools.partial(preprocess_viseme, 
                                   pad_len_in_secs=model_config["paddedAudioLength"], 
                                   blendshapes=source_config["mappings"], 
                                   target_framerate=model_config["frameRate"])

training_data = VisemeDataset("./data/training/speaker_3/", 
                              process_audio, \
                              process_viseme, 
                              num_ipa_symbols=157,
                             text_pad_len=text_pad_len)
test_data = VisemeDataset("./data/test/speaker_3/", 
                              process_audio, \
                              process_viseme,
                              num_ipa_symbols=157,
                              text_pad_len=text_pad_len
                             )
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)

input_dim = int(model_config["windowLength"] / model_config["hopSize"] * model_config["numMels"])

model_config
input_dim


['MouthClose', 'MouthFunnel', 'MouthPucker', 'JawOpen', 'EyeBlinkLeft', 'EyeBlinkRight', 'EyeSquintLeft', 'EyeSquintRight', 'BrowDownLeft', 'BrowDownRight', 'MouthUpperUpLeft', 'MouthUpperUpRight', 'MouthLowerDownLeft', 'MouthLowerDownRight', 'MouthRollUpper']


600

In [4]:
class SeparableConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, bias=False):
        super(SeparableConv1d, self).__init__()
        self.depthwise = nn.Conv1d(in_channels, in_channels, kernel_size=kernel_size,
                               groups=in_channels, bias=bias, padding=1)
        self.pointwise = nn.Conv1d(in_channels, out_channels,
                               kernel_size=1, bias=bias)
    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out
    
class Conv1dModel(nn.Module):
    def __init__(self, 
                 seq_length=None, 
                 input_dim=None, 
                 text_dim=None, 
                 text_embedding_dim=512, 
                 text_padding_idx=None,
                 num_visemes=4, 
                 ks=64):
        super(Conv1dModel, self).__init__()
        #self.text_padding_idx = text_padding_idx;
        #self.text_embedding = nn.Sequential(
        #        nn.Embedding(text_dim,text_embedding_dim, padding_idx=self.text_padding_idx),
        #        nn.Linear(text_embedding_dim, input_dim),
        #        nn.ReLU()
        #)
                
        self.conv1 = nn.Sequential(
            SeparableConv1d(seq_length,seq_length,ks),
            nn.ReLU(),
        )
        self.conv2 =  nn.Sequential(
            SeparableConv1d(seq_length,seq_length,int(ks/2)),
            nn.ReLU(),
        )
        #self.text_attention = nn.MultiheadAttention(input_dim, 1, batch_first=True)
        #self.dropout = torch.nn.Dropout()
        #self.attention = nn.MultiheadAttention(1951, 1, batch_first=True)
        self.linear_out = nn.Linear(510, num_visemes)
        #nn.ModuleList([
            
        #])
        
    def forward(self, audio_feats, text_feats):
        #audio_feats = torch.transpose(audio_feats,1,2)
        #text_out = self.text_embedding(text_feats)        
        
        #text_out_attn, text_out_attn_weights = self.text_attention(audio_feats,audio_feats,text_out)
        audio_out = self.conv1(audio_feats)
        
        audio_out = self.conv2(audio_out)
        
        #out = text_out_attn + audio_out
        out = audio_out
        
        #print(out.size())
        #out = self.dropout(out)
        #return torch.squeeze(torch.stack([v(out) for v in self.linear_out],dim=3))
        return self.linear_out(out)

#20x199x5040
model = Conv1dModel(
    seq_length=seq_length,
    input_dim=input_dim, 
    num_visemes=len(model_config["targetNames"]),
    text_dim=training_data.num_ipa_symbols,
    text_padding_idx=training_data.pad_token).to(device)
batch = iter(train_dataloader)
audio_features, text_features, train_mask, train_labels, train_files = next(batch, (None,None,None,None,None))
model.forward(audio_features.to(device), text_features.to(device))
train_labels.size()
train_labels[0,:,:]


Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)


tensor([[0.1718, 0.1318, 0.1247,  ..., 0.1289, 0.1453, 0.0825],
        [0.2337, 0.1899, 0.1399,  ..., 0.1447, 0.1613, 0.0995],
        [0.1244, 0.2427, 0.1072,  ..., 0.1623, 0.1755, 0.0690],
        ...,
        [0.1718, 0.1318, 0.1247,  ..., 0.1289, 0.1453, 0.0825],
        [0.1718, 0.1318, 0.1247,  ..., 0.1289, 0.1453, 0.0825],
        [0.1718, 0.1318, 0.1247,  ..., 0.1289, 0.1453, 0.0825]])

In [5]:
train_files

('data/training/speaker_3/2.csv',
 'data/training/speaker_3/33.csv',
 'data/training/speaker_3/438.csv',
 'data/training/speaker_3/123.csv',
 'data/training/speaker_3/315.csv')

In [ ]:

#model = Conv2dModel(hidden_size=256, input_dim=input_dim, num_visemes=len(model_config["targetNames"])).to(device)


learning_rate = 0.0005
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
num_steps = 100000
print_loss_every = 100
eval_every = 500

batch = iter(train_dataloader)
accum_loss = 0
audio_features, text_features, train_mask, train_labels, train_files = next(batch, (None,None,None,None,None))
for t in range(num_steps):
    optimizer.zero_grad()
#    audio_features, text_features, train_mask, train_labels, train_files = next(batch, (None,None,None,None,None))
    
#    if audio_features is None:
#        batch = iter(train_dataloader)
#        audio_features, text_features, train_mask, train_labels, _ = next(batch)
            
    preds = model(audio_features.to(device),text_features.to(device)) 
    
    #loss = torch.nn.functional.huber_loss(preds, train_labels.to(device) )
    loss = torch.nn.functional.mse_loss(preds, train_labels.to(device))
    if len(preds.shape) < 3:
        preds = torch.unsqueeze(preds, 0)
    
    #for i in range(preds.shape[1] - 1):
    #    cosine_loss = torch.nn.functional.cosine_embedding_loss(
    #        preds[:,i,:], 
    #        preds[:,i+1,:], 
    #        (torch.ones(preds.shape[0])
    #    ).to(device))
    #    loss += cosine_loss
    accum_loss += loss.item()
    if t % print_loss_every == 0:
        print(f"Step {t} Avg loss: {accum_loss / print_loss_every}")
        accum_loss = 0
    loss.backward()
    optimizer.step()
        
    if t > 0 and t % eval_every == 0:
        accum_loss = 0
        for audio_feats, text_feats, test_mask, test_labels, _ in iter(test_dataloader):
            y = test_labels.to(device)
            preds = model(audio_feats.to(device), text_feats.to(device))
            accum_loss += torch.nn.functional.mse_loss(preds, y).item()

        print(f"Test loss {accum_loss}")
        accum_loss = 0
    
#pred_probab = nn.Softmax(dim=1)(logits)
#y_pred = pred_probab.argmax(1)
#print(f"Predicted class: {y_pred}")199

Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Step 0 Avg loss: 0.00036869656294584273
Step 100 Avg loss: 0.0064223116566427055
Step 200 Avg loss: 0.00251761358929798
Step 300 Avg loss: 0.0015828497451730072
Step 400 Avg loss: 0.0012265170610044152
Step 500 Avg loss: 0.0010307991382433102
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded shape
(220500,)
pad_len is 199
(199, 63)
Post-padded sha

In [ ]:
preds

In [ ]:
audio_feats, text_feats, _, _, files = next(iter(test_dataloader))
export_y_batch = model(audio_feats.to(device), text_feats.to(device))
export_y = export_y_batch[0,:,:]
print(files[0])
print(export_y)

header = "Timecode,BlendShapeCount,eyeBlinkRight,eyeLookDownRight,eyeLookInRight,eyeLookOutRight,eyeLookUpRight,eyeSquintRight,eyeWideRight,eyeBlinkLeft,eyeLookDownLeft,eyeLookInLeft,eyeLookOutLeft,eyeLookUpLeft,eyeSquintLeft,eyeWideLeft,jawForward,jawRight,jawLeft,jawOpen,mouthClose,mouthFunnel,mouthPucker,mouthRight,mouthLeft,mouthSmileRight,mouthSmileLeft,mouthFrownRight,mouthFrownLeft,mouthDimpleRight,mouthDimpleLeft,mouthStretchRight,mouthStretchLeft,mouthRollLower,mouthRollUpper,mouthShrugLower,mouthShrugUpper,mouthPressRight,mouthPressLeft,mouthLowerDownRight,mouthLowerDownLeft,mouthUpperUpRight,mouthUpperUpLeft,browDownRight,browDownLeft,browInnerUp,browOuterUpRight,browOuterUpLeft,cheekPuff,cheekSquintRight,cheekSquintLeft,noseSneerRight,noseSneerLeft,tongueOut,HeadYaw,HeadPitch,HeadRoll,LeftEyeYaw,LeftEyePitch,LeftEyeRoll,RightEyeYaw,RightEyePitch,RightEyeRoll".split(',')
selected_output_indices = [header.index(x[0].lower() + x[1:]) for x in source_config["mappings"].keys()]
num_visemes = len(source_config["mappings"])
with open("output/prediction.csv", "w") as outfile:
    outfile.write(",".join(header) + "\n")
    timer_ms = 0
    for t in range(export_y.shape[1]):
        output = [str(0)] * len(header)
        second = str(int(timer_ms // 1000)).zfill(2)
        frame = (timer_ms % 1000) * model_config["frameRate"] / 1000
        output[0] = f"00:00:{second}:{frame}"
        for viseme in range(num_visemes): 
            output[selected_output_indices[viseme]] = str(export_y[viseme,t].item())
        timer_ms += (1 / model_config["frameRate"]) * 1000
        outfile.write(",".join(output) + "\n")

In [ ]:
new_header = "Timecode,BlendShapeCount,EyeBlinkLeft,EyeLookDownLeft,EyeLookInLeft,EyeLookOutLeft,EyeLookUpLeft,EyeSquintLeft,EyeWideLeft,EyeBlinkRight,EyeLookDownRight,EyeLookInRight,EyeLookOutRight,EyeLookUpRight,EyeSquintRight,EyeWideRight,JawForward,JawRight,JawLeft,JawOpen,MouthClose,MouthFunnel,MouthPucker,MouthRight,MouthLeft,MouthSmileLeft,MouthSmileRight,MouthFrownLeft,MouthFrownRight,MouthDimpleLeft,MouthDimpleRight,MouthStretchLeft,MouthStretchRight,MouthRollLower,MouthRollUpper,MouthShrugLower,MouthShrugUpper,MouthPressLeft,MouthPressRight,MouthLowerDownLeft,MouthLowerDownRight,MouthUpperUpLeft,MouthUpperUpRight,BrowDownLeft,BrowDownRight,BrowInnerUp,BrowOuterUpLeft,BrowOuterUpRight,CheekPuff,CheekSquintLeft,CheekSquintRight,NoseSneerLeft,NoseSneerRight,TongueOut,HeadYaw,HeadPitch,HeadRoll,LeftEyeYaw,LeftEyePitch,LeftEyeRoll,RightEyeYaw,RightEyePitch,RightEyeRoll"
remap = {h:(h[0].lower() + h[1:]) if h not in ["Timecode","BlendShapeCount","HeadYaw","HeadPitch","HeadRoll","LeftEyeYaw","LeftEyePitch","LeftEyeRoll","RightEyeYaw","RightEyePitch","RightEyeRoll"]  else h for h in new_header.split(",") }
for oh in remap.values():
    if oh not in header:
        print(oh)

def new_to_old(csv_df):
    return csv_df.rename(columns=remap)
df = preprocess_viseme("data/training/speaker_1/20210824_1/61.csv", pad_len_in_secs=model_config["paddedAudioLength"], 
                                   target_framerate=model_config["frameRate"])
df = new_to_old(df)
#cols = [c for c in df.columns if c not in ["jawOpen", "mouthClose", "Timecode"]]
#df[cols] = 0
df = df[df["Timecode"] != 0]
df.to_csv("original.csv", index=False)
#new_to_old(df)[header].to_csv("original.csv", index=False)
df

In [ ]:
df = preprocess_viseme("data/training/speaker_1/20210824_1/61.csv", pad_len_in_secs=model_config["paddedAudioLength"], 
                                   target_framerate=
                       model_config["frameRate"])
print(list(source_config["mappings"].keys()))
print(list(source_config["mappings"].values()))
df = df[list(source_config["mappings"].keys())]
df.columns = list(source_config["mappings"].values())
df = df.iloc[::int(59.97 / 29.98)]


df.to_csv("original.csv", index=False)
#model_config
df

In [ ]:
df.iloc[::int(59.97 / 29.98)]



In [ ]:
torch.save(model, "output/conv2d.torch")

In [ ]:
model = torch.load("output/conv2d.torch",map_location=torch.device('cpu'))
import torch.onnx 

# set the model to inference mode 
model.eval() 

# Let's create a dummy input tensor  
dummy_input = torch.randn(1, input_dim, 119,  requires_grad=False)  

# Export the model   
torch.onnx.export(model,         # model being run 
     dummy_input,       # model input (or a tuple for multiple inputs) 
     "output/conv2d.onnx",       # where to save the model  
     export_params=True,  # store the trained parameter weights inside the model file 
     opset_version=10,    # the ONNX version to export the model to 
     do_constant_folding=True,  # whether to execute constant folding for optimization 
     #input_names = ['modelInput'],   # the model's input names 
     #output_names = ['modelOutput'], # the model's output names 
     dynamic_axes={'modelInput' : {0 : 'batch_size'},    # variable length axes 
    'modelOutput' : {0 : 'batch_size'}}
                 ) 


In [ ]:
import onnx

import onnxruntime
model_onnx = onnx.load('output/conv2d.onnx')


from onnx_tf.backend import prepare
tf_rep = prepare(model_onnx)
tf_rep.export_graph('./output/tf_model')




In [ ]:
import tensorflow_model_optimization as tfmot

quant_aware_model = tfmot.quantization.keras.quantize_model(tf_rep)
quant_aware_model.summary()

In [ ]:
import tensorflow as tf

converter = tf.lite.TFLiteConverter.from_saved_model("./output/tf_model")
print("Built converter")

converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
converter.allow_custom_ops=False
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.experimental_enable_resource_variables = True

converter.experimental_new_converter =True
tflite_model = converter.convert()
print("Converted")

# Save the model
with open("./output/conv2d.tflite", 'wb') as f:
    f.write(tflite_model)

In [ ]:
interpreter = tf.lite.Interpreter(model_path="bilstm.tflite")
interpreter.allocate_tensors()
    
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']
output_details

In [ ]:
csv = pd.read_csv("data/training/speaker_1/20210824_1/61.csv")
columns = [x for x in list(csv.columns) if "Eye" not in x]
columns.remove("Timecode")
columns.remove("BlendShapeCount")
csv[columns].var().sort_values()
#df = preprocess_viseme("data/training/speaker_1/20210824_1/61.csv", pad_len_in_secs=pad_len_in_secs, 
#                                   resample_to=target_framerate, blendshapes=["MouthClose","MouthFunnel"])
#df.shape
#[df.iloc[0]["EyeLookInLeft"]]
    #csv[columns] = pd.np.digitize(csv[columns], np.linspace(0,1,11))
    
    #split = csv["Timecode"].str.split(':')
    #minute = split.str[1].astype(int)
    #second = split.str[2].astype(int)
    #frame = split.str[3].astype(float)
    #minute -= minute[0]
    #ms
    #step = minute * 60 + second
    #csv["step"] = step
    #return csv.drop_duplicates(["step"])[["step", "MouthClose","MouthFunnel","MouthPucker","JawOpen"]]
    
# if we want to use softmax across each blendshape as a one-hot
    #return np.reshape(vals, (vals.shape[0], vals.shape[1], 1))
    #one_hot = np.zeros((vals.shape[0], vals.shape[1], 11, 1))
    #oh = np.eye(11)
    #for row in range(vals.shape[0]):
    #    for t in range(vals.shape[1]):
    #        one_hot[row, :, :, 0] = np.eye(11)[int(vals[row,t])-1]
    #return one_hot